<a href="https://colab.research.google.com/github/viniciuspxc/Projeto_02_LogiComp/blob/main/LogiComp_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Declaração Python e PySat

In [ ]:
!python -V

Python 3.7.13


In [ ]:
!pip install python-sat[pblib,aiger]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pysat.formula import IDPool
from pysat.formula import CNF
from pysat.solvers import Cadical
Atom = IDPool()

# Leitura de dados (Pandas)

In [ ]:
import pandas as pd

In [ ]:
dados= pd.read_csv("/content/column_bin_3a_3p.csv",sep=',')
dados.index += 1
dados

,PI <= 42.09,PI <= 70.62,GS <= 57.55,P
1,0,0,1,1
2,0,0,0,1
3,0,1,0,0


In [ ]:
print("Pacientes: "+str(dados.shape[0]))
print("\nAtributos: "+str(list(dados)))
print("\nMatriz dos valores: "+str([tuple(x) for x in dados.values]))

Pacientes: 3

Atributos: ['PI <= 42.09', 'PI <= 70.62', 'GS <= 57.55', 'P']

Matriz dos valores: [(0, 0, 1, 1), (0, 0, 0, 1), (0, 1, 0, 0)]


# Quantidade de Regras

In [ ]:
m = 1

# Restrições

## Funções Auxiliares

In [ ]:
def lerdados(dados):
    pacientes=dados.shape[0]
    valor_atributos = [tuple(x) for x in dados.values]
    atributos = list(dados)
    return pacientes,atributos,valor_atributos

def watritbuto(a, i, t):
    return (Atom.id(a + '-' + str(i + 1) + '-' + t))

def wregra(i, j):
    return Atom.id('C-' + str(i + 1) + '-' + str(j + 1))

def pretty_formula_printer(formula):
  for clause in formula:
    for literal in clause:
      if literal > 0:
        print(Atom.obj(literal), ' ',  end = '')
      else:
        print('Not', Atom.obj(literal*-1), ' ',  end = '')
    print('')

## Restrição 1


Para cada atributo e cada regra, temos exatamente uma das três possibilidades: o atributo aparece positivamente na regra, o atributo aparece negativamente na regra ou o atributo não aparece na regra.

In [ ]:
def r1(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    lista3 = []
    formula = []
    for i in range(m):
        for a in atributos:
            if a != 'P':
                lista=[]
                lista.append(watritbuto(a, i, 'p'))
                lista.append(watritbuto(a, i, 'n'))
                lista.append(watritbuto(a, i, 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'p'))
                lista.append(-watritbuto(a, i, 'n'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'p'))
                lista.append(-watritbuto(a, i, 's'))
                lista2.append(lista)

                lista=[]
                lista.append(-watritbuto(a, i, 'n'))
                lista.append(-watritbuto(a, i, 's'))
                lista2.append(lista)

    formula = lista2
    return formula

f1=r1(m,dados)
print(f1)
pretty_formula_printer(f1)

[[2, 5, 6], [-2, -5], [-2, -6], [-5, -6], [3, 7, 8], [-3, -7], [-3, -8], [-7, -8], [4, 9, 10], [-4, -9], [-4, -10], [-9, -10]]
PI <= 42.09-1-p  PI <= 42.09-1-n  PI <= 42.09-1-s  
Not PI <= 42.09-1-p  Not PI <= 42.09-1-n  
Not PI <= 42.09-1-p  Not PI <= 42.09-1-s  
Not PI <= 42.09-1-n  Not PI <= 42.09-1-s  
PI <= 70.62-1-p  PI <= 70.62-1-n  PI <= 70.62-1-s  
Not PI <= 70.62-1-p  Not PI <= 70.62-1-n  
Not PI <= 70.62-1-p  Not PI <= 70.62-1-s  
Not PI <= 70.62-1-n  Not PI <= 70.62-1-s  
GS <= 57.55-1-p  GS <= 57.55-1-n  GS <= 57.55-1-s  
Not GS <= 57.55-1-p  Not GS <= 57.55-1-n  
Not GS <= 57.55-1-p  Not GS <= 57.55-1-s  
Not GS <= 57.55-1-n  Not GS <= 57.55-1-s  


## Restrição 2

Cada regra deve ter algum atributo aparecendo nela.

In [ ]:
def r2(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for i in range(m):
        lista = []
        for a in atributos:
            if a != 'P':
                lista.append(-watritbuto(a, i, 's'))
        lista2.append(lista)
    formula = lista2
    return formula

f2=r2(m,dados)
print(f2)
pretty_formula_printer(f2)

[[-6, -8, -10]]
Not PI <= 42.09-1-s  Not PI <= 70.62-1-s  Not GS <= 57.55-1-s  


## Restrição 3

 Para cada paciente sem patologia e cada regra, algum atributo do paciente não pode ser aplicado à regra.

In [ ]:
def r3(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        if valor_atributos[j][atributos.index('P')] == 0:
            for i in range(m):
                lista = []
                for a in atributos:
                    if a != 'P':
                        if valor_atributos[j][atributos.index(str(a))] == 1: 
                            lista.append(watritbuto(a, i, 'n'))
                        else:
                            lista.append(watritbuto(a, i, 'p'))
                lista2.append(lista)
    formula = lista2
    return formula

f3=r3(m,dados)
print(f3)
pretty_formula_printer(f3)

[[2, 7, 4]]
PI <= 42.09-1-p  PI <= 70.62-1-n  GS <= 57.55-1-p  


## Restrição 4

Para cada paciente com patologia, cada regra e cada atributo, se o atributo do paciente não se aplicar ao da regra, então a regra não cobre esse paciente.

In [ ]:
def r4(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        if valor_atributos[j][atributos.index('P')] == 1:
            for i in range(m):
                lista = []
                for a in atributos:
                    if a != 'P':
                        if valor_atributos[j][atributos.index(str(a))] == 1:
                            lista.append(-watritbuto(a, i, 'n'))
                            lista.append(-wregra(i, j))
                        else:
                            lista.append(-watritbuto(a, i, 'p'))
                            lista.append(-wregra(i, j))
                lista2.append(lista)
    formula = lista2
    return formula

f4=r4(m,dados)
print(f4)
pretty_formula_printer(f4)

[[-2, -11, -3, -11, -9, -11], [-2, -12, -3, -12, -4, -12]]
Not PI <= 42.09-1-p  Not C-1-1  Not PI <= 70.62-1-p  Not C-1-1  Not GS <= 57.55-1-n  Not C-1-1  
Not PI <= 42.09-1-p  Not C-1-2  Not PI <= 70.62-1-p  Not C-1-2  Not GS <= 57.55-1-p  Not C-1-2  


## Restrição 5

Cada paciente com patologia deve ser coberto por alguma das regras.

In [ ]:
def r5(m, dados):
    pacientes,atributos,valor_atributos = lerdados(dados)
    lista = []
    lista2 = []
    for j in range(pacientes):
        lista = []
        if valor_atributos[j][atributos.index('P')] == 1:
          for i in range(m):
                lista.append(wregra(i, j))
          lista2.append(lista)
    formula = lista2
    return formula

f5=r5(m,dados)
print(f5)
pretty_formula_printer(f5)

[[11], [12]]
C-1-1  
C-1-2  


# Execução

In [ ]:
fr = r1(m,dados)+r2(m,dados)+r3(m,dados)+r4(m,dados)+r5(m,dados)
print(fr)
#solver = Cadical()
#solver.append_formula(final_formula)

[[2, 5, 6], [-2, -5], [-2, -6], [-5, -6], [3, 7, 8], [-3, -7], [-3, -8], [-7, -8], [4, 9, 10], [-4, -9], [-4, -10], [-9, -10], [-6, -8, -10], [2, 7, 4], [-2, -11, -3, -11, -9, -11], [-2, -12, -3, -12, -4, -12], [11], [12]]


# Fim